In [1]:
import rclpy
import rclpy.logging
from rclpy.node import Node
from actuator_msgs.msg import Actuators
from std_msgs.msg import Header, Int16

from rclpy.qos import QoSProfile, ReliabilityPolicy, HistoryPolicy, DurabilityPolicy
from nav_msgs.msg import Odometry

import numpy as np
import math
from collections import deque

In [2]:
import pandas as pd
from pydrake.all import LinearQuadraticRegulator

In [3]:
mass = 2 + 0.016076923076923075 * 4
mass

2.0643076923076924

In [4]:
Ixx_B = 0.02166666666666667
Iyy_B = 0.02166666666666667
Izz_B = 0.04000000000000001

Ixx_P = 3.8464910483993325e-07
Iyy_P = 2.6115851691700804e-05
Izz_P = 2.649858234714004e-05

Ixx_t = Ixx_B + 4 * Ixx_P
Iyy_t = Iyy_B + 4 * Iyy_P
Izz_t = Izz_B + 4 * Izz_P


In [5]:
l = 0.25

In [6]:
global KF, Y
KF = 0.0
Y = 0.0

In [7]:
moment_constant = 0.016

In [8]:
def quaternion_to_rpy(w, x, y, z):
    """
    Convert quaternion to roll, pitch, and yaw (RPY) angles.
    
    Parameters:
    - x: float, x component of the quaternion
    - y: float, y component of the quaternion
    - z: float, z component of the quaternion
    - w: float, w component of the quaternion
    
    Returns:
    - roll: float, roll angle in radians
    - pitch: float, pitch angle in radians
    - yaw: float, yaw angle in radians
    """
    
    # Roll (x-axis rotation)
    sinr_cosp = 2 * (w * x + y * z)
    cosr_cosp = 1 - 2 * (x * x + y * y)
    roll = math.atan2(sinr_cosp, cosr_cosp)

    # Pitch (y-axis rotation)
    sinp = 2 * (w * y - z * x)
    if abs(sinp) >= 1:
        pitch = math.copysign(math.pi / 2, sinp)  # Use 90 degrees if out of range
    else:
        pitch = math.asin(sinp)

    # Yaw (z-axis rotation)
    siny_cosp = 2 * (w * z + x * y)
    cosy_cosp = 1 - 2 * (y * y + z * z)
    yaw = math.atan2(siny_cosp, cosy_cosp)

    return roll, pitch, yaw

In [26]:
class CommandPublisherNode(Node):
    def __init__(self):
        super().__init__('command_publisher_node')
        self.logger = self.get_logger()
        self.motor_command_pub = self.create_publisher(Actuators, '/x500/command/motor_speed', 10)
        
        qos_profile = QoSProfile(
            reliability=ReliabilityPolicy.BEST_EFFORT,
            durability=DurabilityPolicy.VOLATILE,
            history=HistoryPolicy.KEEP_LAST,
            depth=5
        )
        
        # Create subscription with proper QoS
        self.subscription = self.create_subscription(
            Odometry,
            '/x500/odometry',
            self.odometry_callback,
            qos_profile)
        self.initialize_params()
        
    def initialize_params(self):        
        self.actuators_msg = Actuators()
        self.actuators_msg.header = Header()
        self.actuators_msg.header.stamp = self.get_clock().now().to_msg()
        self.actuators_msg.header.frame_id = ''
        self.actuators_msg.velocity = [float(0), float(0), float(0), float(0)]
                
        # Initialize class attributes
        self.clock_time = self.get_clock().now()
        self.x = self.y = self.z = 0.0
        self.vx = self.vy = self.vz = 0.0
        self.prev_vx = self.prev_vy = self.prev_vz = 0.0
        self.ax = self.ay = self.az = 0.0
        self.roll = self.pitch = self.yaw = 0.0
        
        # Previous velocities for acceleration calculation
        self.prev_vx = self.prev_vy = self.prev_vz = 0.0
        
        # Create a rate
        self.frequency = 100
        self.rate = self.create_rate(self.frequency) 
        
        # motor commands
        self.command_values = [0.0, 0.0, 0.0, 0.0]
        self.to_measure_Y = False      #torque coeff
        self.to_measure_KF = False      #force coeff
        self.measurement_queue = deque(maxlen=10)
        self.counter = 0
        # for force coeff
        self.avg_acc_error_tol = 0.01
        self.avg__w__error_tol = 0.01
        self.prev_value = 0.0
        self.to_stop_spin = False
        
    def calculate_accelerations(self):
        """Calculate accelerations from velocity changes"""
        curr_time = self.get_clock().now()
        del_t = curr_time.seconds_nanoseconds()[0]-self.clock_time.seconds_nanoseconds()[0] + 1e-9 * (curr_time.seconds_nanoseconds()[1]-self.clock_time.seconds_nanoseconds()[1])
        if del_t > 0:
            self.ax = (self.vx - self.prev_vx) / del_t
            self.ay = (self.vy - self.prev_vy) / del_t
            self.az = (self.vz - self.prev_vz) / del_t  
            
    def odometry_callback(self, msg):
        # Position
        self.x = msg.pose.pose.position.x
        self.y = msg.pose.pose.position.y
        self.z = msg.pose.pose.position.z
        # Orientation
        _q = msg.pose.pose.orientation
        self.rpy = list(quaternion_to_rpy(_q.w, _q.x, _q.y, _q.z))
        # Linear velocities
        self.vx = msg.twist.twist.linear.x
        self.vy = msg.twist.twist.linear.y
        self.vz = msg.twist.twist.linear.z
        # Angular velocities
        self.wx = msg.twist.twist.angular.x
        self.wy = msg.twist.twist.angular.y
        self.wz = msg.twist.twist.angular.z
        # self.logger.info(f'{self.z}')
        if self.to_measure_KF:
            self.calculate_accelerations()
            self.average_calculation(self.az, self.avg_acc_error_tol, self.calculate_KF)
            
        if self.to_measure_Y:
            self.average_calculation(self.wz, self.avg__w__error_tol, self.calculate_Y)
                
    def average_calculation(self, x, x_tol, calculate_var):
        if self.counter == 10:
            value = sum(self.measurement_queue) / self.measurement_queue.maxlen
            self.logger.info(f'average acceleration in z = {value}, prev_value: {self.prev_value}')
            self.counter = 0
            self.logger.info(f'{value - self.prev_value}, {x_tol}, {abs(value - self.prev_value) <= x_tol}')
            if(abs(value - self.prev_value) <= x_tol):
                self.logger.info('starting calculation of constants')
                calculate_var(value)
                self.logger.info('Done!!')
            self.prev_value = value
        self.counter += 1
        self.measurement_queue.append(x)
        
    def calculate_KF(self, avg_acc):
        global KF
        KF = mass * (avg_acc + 9.8) / 4 / self.command_values[0]**2
        self.logger.info(f'Calculated Value of Force coeff of a single motor: {KF}')
        self.to_stop_spin = True
        
    def calculate_Y(self, avg_w):
        global Y
        W2 = [v**2 for v in self.command_values]
        torque = KF * moment_constant * (W2[0] + W2[1] - W2[2] - W2[3])
        Y = abs(torque) / abs(avg_w)
        self.logger.info(f'Calculated Value of drag coeff of the drone: {Y}')
        self.to_stop_spin = True
    
    def command_pub(self, cmd: list):
        assert len(cmd) == 4
        self.command_values = cmd
        self.actuators_msg.header.stamp = self.get_clock().now().to_msg()
        self.actuators_msg.velocity = [float(v) for v in cmd]
        self.motor_command_pub.publish(self.actuators_msg)
        
        

To measure Force coeff, make sure the drone is launched in gazebo with a prismatic joint in z direction.

In [27]:
rclpy.init()
commander = CommandPublisherNode()

In [ ]:
commander.to_measure_KF = True

In [11]:
thrust_value = 550
commander.command_pub([thrust_value for i in range(4)])

Make sure the propellers are rotating in Gazebo, if yes, then only move forward

In [12]:
while rclpy.ok():
    rclpy.spin_once(commander, timeout_sec=0.01)
    if commander.to_stop_spin:
        break
commander.initialize_params()
# rclpy.shutdown()
# commander.destroy_node()

[INFO] [1732487467.588164345] [command_publisher_node]: average acceleration in z = 0.12474626061999183, prev_value: 0.0
[INFO] [1732487467.597091814] [command_publisher_node]: 0.12474626061999183, 0.01, False
[INFO] [1732487467.748585485] [command_publisher_node]: average acceleration in z = 0.18670938380467234, prev_value: 0.12474626061999183
[INFO] [1732487467.752735387] [command_publisher_node]: 0.0619631231846805, 0.01, False
[INFO] [1732487467.952464792] [command_publisher_node]: average acceleration in z = 0.20037687816682692, prev_value: 0.18670938380467234
[INFO] [1732487467.955892135] [command_publisher_node]: 0.013667494362154586, 0.01, False
[INFO] [1732487468.156128558] [command_publisher_node]: average acceleration in z = 0.20453664016658119, prev_value: 0.20037687816682692
[INFO] [1732487468.164692196] [command_publisher_node]: 0.004159761999754263, 0.01, True
[INFO] [1732487468.167879139] [command_publisher_node]: starting calculation of constants
[INFO] [1732487468.175

In [28]:
KF

1.70681338382397e-05

To measure gamma coeff, make sure the drone is launched in gazebo with a revolute joint in z direction.

In [29]:
commander.to_measure_Y = True

In [30]:
thrust_value = 200
commander.command_pub([thrust_value if i<2 else 0.0 for i in range(4)])

Make sure the propellers are rotating in Gazebo, if yes, then only move forward

In [31]:
while rclpy.ok():
    rclpy.spin_once(commander, timeout_sec=0.01)
    if commander.to_stop_spin:
        break
commander.initialize_params()

[INFO] [1732487837.013559969] [command_publisher_node]: average acceleration in z = -11.200526896759412, prev_value: 0.0
[INFO] [1732487837.020249682] [command_publisher_node]: -11.200526896759412, 0.01, False
[INFO] [1732487837.217858607] [command_publisher_node]: average acceleration in z = 20.215399603879554, prev_value: -11.200526896759412
[INFO] [1732487837.226601016] [command_publisher_node]: 31.415926500638967, 0.01, False
[INFO] [1732487837.421332219] [command_publisher_node]: average acceleration in z = -11.20052696695402, prev_value: 20.215399603879554
[INFO] [1732487837.430857809] [command_publisher_node]: -31.415926570833577, 0.01, False
[INFO] [1732487837.621857864] [command_publisher_node]: average acceleration in z = -11.200527001586515, prev_value: -11.20052696695402
[INFO] [1732487837.630596365] [command_publisher_node]: -3.4632494205766307e-08, 0.01, True
[INFO] [1732487837.635279312] [command_publisher_node]: starting calculation of constants
[INFO] [1732487837.64197

In [32]:
rclpy.shutdown()
commander.destroy_node()

In [33]:
Y

0.0019505520865091648